# Stair detection

In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

%matplotlib ipympl

Function to display images inline

In [2]:
def imshow(image, figsize):
    plt.figure(figsize=figsize)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

### Reading images and detecting edges

In [3]:
img = cv.imread("testImages\stairDetectionTestImages\stair.png")
blur = cv.GaussianBlur(img, (3,3), 0)
grey = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
edge = cv.Canny(grey, 100, 150, apertureSize=3)

imshow(edge, (7,7))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fitting lines to edges

In [4]:
lines = cv.HoughLines(edge, 1, np.pi/180, 110)
print(lines.shape)
print(lines)

(27, 1, 2)
[[[ 6.3300000e+02  1.5707964e+00]]

 [[ 3.0200000e+02  1.5707964e+00]]

 [[ 6.9500000e+02  1.5882496e+00]]

 [[ 4.2200000e+02  1.5707964e+00]]

 [[ 3.2900000e+02  1.5707964e+00]]

 [[ 6.2500000e+02  1.5707964e+00]]

 [[ 4.1600000e+02  1.5882496e+00]]

 [[ 2.3700000e+02  1.3962634e-01]]

 [[ 5.7100000e+02  1.5707964e+00]]

 [[ 7.0900000e+02  1.6057029e+00]]

 [[ 5.6800000e+02  1.5707964e+00]]

 [[ 5.0100000e+02  1.6057029e+00]]

 [[ 2.4600000e+02  1.5707964e-01]]

 [[ 2.3100000e+02  1.2217305e-01]]

 [[ 4.5900000e+02  1.5882496e+00]]

 [[ 3.5000000e+02  1.5882496e+00]]

 [[ 4.5700000e+02  1.5882496e+00]]

 [[ 3.8500000e+02  1.5707964e+00]]

 [[ 6.2800000e+02  1.5882496e+00]]

 [[ 4.6200000e+02  1.5707964e+00]]

 [[ 5.0600000e+02  1.5882496e+00]]

 [[ 6.9100000e+02  1.6057029e+00]]

 [[ 5.6200000e+02  1.5882496e+00]]

 [[ 3.0500000e+02  1.5707964e+00]]

 [[-2.7200000e+02  2.9146998e+00]]

 [[ 7.0200000e+02  1.5707964e+00]]

 [[ 5.0300000e+02  1.5882496e+00]]]


### View lines

In [5]:
for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    # x1 stores the rounded off value of (r * cos(theta) - 1000 * sin(theta))
    x1 = int(x0 + 1000 * (-b))
    # y1 stores the rounded off value of (r * sin(theta)+ 1000 * cos(theta))
    y1 = int(y0 + 1000 * (a))
    # x2 stores the rounded off value of (r * cos(theta)+ 1000 * sin(theta))
    x2 = int(x0 - 1000 * (-b))
    # y2 stores the rounded off value of (r * sin(theta)- 1000 * cos(theta))
    y2 = int(y0 - 1000 * (a))
    
    cv.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)    


imshow(img, (7,7))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Stair Angles

In [6]:
# Image refresh
image = cv.imread("testImages\stairDetectionTestImages\stair.png")

### Find stair side angles

In [7]:
def returnStairSides(image, lines):
    """Return angles of stair sides in image"""
    angleL = []
    angleR = []
    image = image.copy()

    for line in lines:
        rho,theta = line[0]
        if 0.1<theta<0.8:
            print(len(angleL),'Angle @ Left = ', theta)

            a = np.cos(theta)
            b = np.sin(theta)

            x0 = a * rho
            y0 = b * rho

            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))

            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))

            cv.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            angleL.append(theta)

        elif 2.4<theta<3:
            print(len(angleR),'Angle @ Right = ', theta)

            a = np.cos(theta)
            b = np.sin(theta)

            x0 = a * rho
            y0 = b * rho

            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))

            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))

            cv.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            angleR.append(theta)

    angleL = np.mean(angleL)
    angleR = np.mean(angleR)

    return angleL, angleR, image

angleL, angleR, sideLineImage = returnStairSides(image, lines)
      
plt.figure(figsize=(7,7))
plt.imshow(cv.cvtColor(sideLineImage, cv.COLOR_BGR2RGB))

0 Angle @ Left =  0.13962634
1 Angle @ Left =  0.15707964
2 Angle @ Left =  0.12217305
0 Angle @ Right =  2.9146998


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Keeping Y-axis as base, angle is measured anti-clockwise. 

In [8]:
print("Stair side angle left = ", angleL)
print("Stair side angle light = ", angleR)

Stair side angle left =  0.13962634
Stair side angle light =  2.9146998


### Find horizontal step angles

In [9]:
def returnStairSteps(image, lines, threshold=30):
    """Return angles of stair sides in image"""
    angleV = []
    image = image.copy()
    
    threshold = np.array([90+threshold, 90-threshold])*np.pi/180

    for line in lines:
        rho,theta = line[0]
        if threshold[1]<theta<threshold[0]:
            print(len(angleV),'Angle @ horizontal = ', theta)

            a = np.cos(theta)
            b = np.sin(theta)

            x0 = a * rho
            y0 = b * rho

            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))

            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))

            cv.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            angleV.append(theta)
        
    angleV = np.mean(angleV)

    return angleV, image

angleV, stepLineImage = returnStairSteps(image, lines)

imshow(stepLineImage, (7,7))

0 Angle @ horizontal =  1.5707964
1 Angle @ horizontal =  1.5707964
2 Angle @ horizontal =  1.5882496
3 Angle @ horizontal =  1.5707964
4 Angle @ horizontal =  1.5707964
5 Angle @ horizontal =  1.5707964
6 Angle @ horizontal =  1.5882496
7 Angle @ horizontal =  1.5707964
8 Angle @ horizontal =  1.6057029
9 Angle @ horizontal =  1.5707964
10 Angle @ horizontal =  1.6057029
11 Angle @ horizontal =  1.5882496
12 Angle @ horizontal =  1.5882496
13 Angle @ horizontal =  1.5882496
14 Angle @ horizontal =  1.5707964
15 Angle @ horizontal =  1.5882496
16 Angle @ horizontal =  1.5707964
17 Angle @ horizontal =  1.5882496
18 Angle @ horizontal =  1.6057029
19 Angle @ horizontal =  1.5882496
20 Angle @ horizontal =  1.5707964
21 Angle @ horizontal =  1.5707964
22 Angle @ horizontal =  1.5882496


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

If this value is close to 90 degrees, we are looking at the stairs straight on. To improve on this, we could weight the mean more towards the bottom stairs. 

In [10]:
print("Stair step angle horizontal = ", angleV*180/np.pi)

Stair step angle horizontal =  90.6521694569985
